## All Import Modules

In [1]:
import yfinance as yf
import datetime
import numpy as np
from ta import momentum, trend
import pandas as pd


In [2]:
import openpyxl
workbook = openpyxl.Workbook()
sheet = workbook.active

headers = ["Algo Name","SYMBOL","Entry Strick Price", "Entry Position", "Entry Date Time" , "SL","SL(%)","TARGET","TARGET(%)","Exit Position" ,"Exit Date Time","P&L","Exit Status","Tred Time"]
sheet.append(headers)


def Save_Data_in_Sheet(algo_name,symbol,entry_Strick_price,entry_position,entry_datetime,sl,sl_pr,target, target_pr,exit_position,exit_datetime,pnl,exit_status):
    Tred_Time = (exit_datetime - entry_datetime ).total_seconds() / 60
    sl_pr = round(sl_pr,1)
    target_pr = round(target_pr,1)
    sheet.append([algo_name,symbol,entry_Strick_price,entry_position,entry_datetime,sl,sl_pr,target, target_pr,exit_position,exit_datetime,pnl,exit_status,Tred_Time])
    # workbook.save(f"ALGO LOGS.xlsx")
    # print("SAVE DATA")

## MENUPLIATE INPUT DATA

In [3]:
previes_days = 60
def Main_Dataframe(symbol,previes_days):
    # ------------------------------------ DAY TIME FRAME -----------------------------------------------------------
    ticker = yf.Ticker(symbol)
    day_df = ticker.history(period=f"80d", interval="1d")
    
    # Reset the index and convert the 'Date' column to date format
    day_df.reset_index(inplace=True)
    day_df['Date'] = day_df['Date'].dt.date
    
    # Calculate technical indicators
    day_df['Day_10EMA'] = trend.EMAIndicator(day_df['Close'], window=10).ema_indicator()
    
    day_df['Day_RSI'] = momentum.RSIIndicator(day_df['Close'], window=3).rsi()
    day_df['Day_RSI_EMA'] = trend.EMAIndicator(day_df['Day_RSI'], window=6).ema_indicator()

    # -----------ADD EXTRA INDICATORS HERE --------------
    day_df['Day_Trend'] = np.select(
    [day_df['Day_10EMA'] > day_df['Day_10EMA'].shift(1),
     day_df['Day_10EMA'] < day_df['Day_10EMA'].shift(1)],
    ['Uptrend', 'Downtrend'],'Sideways')
    # Rename columns and drop unnecessary columns
    day_df.rename(columns={'Open': 'Day_Open', 'High': 'Day_High', 'Low': 'Day_Low', 'Close': 'Day_Close'}, inplace=True)
    day_df.drop(columns=['Volume', 'Dividends', 'Stock Splits','Day_10EMA','Day_RSI'], inplace=True)
    day_df.dropna(inplace=True)

    # ------------------------------------ 5 MIN TIME FRAME -----------------------------------------------------------
    # previes_days = 5
    df = ticker.history(period=f"{previes_days}d", interval="5m")
    df.reset_index(inplace=True)
    df['Date'] = df['Datetime'].dt.date
    df['Time'] = df['Datetime'].dt.time
    
    df.drop(columns=['Volume', 'Dividends', 'Stock Splits','Datetime'], inplace=True)
    df.dropna(inplace=True)
    # -----------ADD EXTRA INDICATORS HERE --------------
    df['5EMA'] = trend.EMAIndicator(df['Close'], window=5).ema_indicator()
    df['15EMA'] = trend.EMAIndicator(df['Close'], window=15).ema_indicator()
    df['Candle'] = np.where(df['Close'] > df['Open'], 'Green', 'Red')
    
    df['Prev_5EMA'] = df['5EMA'].shift(1)
    df['Prev_15EMA'] = df['15EMA'].shift(1)

    df['RSI'] = momentum.RSIIndicator(df['Close'], window=6).rsi()
    df['RSI_EMA'] = trend.EMAIndicator(df['RSI'], window=12).ema_indicator()
    
   
    short_ema =trend.EMAIndicator(df["Close"], window=5)
    long_ema = trend.EMAIndicator(df["Close"], window=15)
    
    df["5EMA"] = short_ema.ema_indicator()
    df["15EMA"] = long_ema.ema_indicator()
    
    # Determine the perfect trend based on EMA crossovers
    df["Trend"] = "Sideways"  # Default to Sideways
    df.loc[(df["5EMA"] > df["15EMA"]) & (df["5EMA"].shift(1) <= df["15EMA"].shift(1)), "Trend"] = "Uptrend"
    df.loc[(df["5EMA"] < df["15EMA"]) & (df["5EMA"].shift(1) >= df["15EMA"].shift(1)), "Trend"] = "Downtrend"

    first_candle_high = df.groupby('Date')['High'].first()
    first_candle_low = df.groupby('Date')['Low'].first()
    
    # Create a new column for the first candle's high and low
    df['First_Candle_High'] = df['Date'].map(first_candle_high)
    df['First_Candle_Low'] = df['Date'].map(first_candle_low)

    # ------------------------------------ MERGE DATA FRAME -----------------------------------------------------------
    
    merged_df = day_df.merge(df, on='Date', how='inner')
    merged_df.dropna(inplace=True)
    merged_df['DateTime'] = pd.to_datetime(merged_df['Date'].astype(str) + ' ' + merged_df['Time'].astype(str))
    merged_df.drop(['Date', 'Time'], axis=1, inplace=True)
    return round(merged_df,2)
    # return day_df
    # return df

symbol = "SBIN.NS"
data = Main_Dataframe(symbol,previes_days)

In [4]:
all_columsn = ['Day_Open', 'Day_High', 'Day_Low', 'Day_Close', 'Day_RSI_EMA',
        'Open', 'High', 'Low', 'Close', '5EMA', '15EMA',
       'Prev_5EMA', 'Prev_15EMA', 'RSI', 'RSI_EMA', 'First_Candle_High', 'First_Candle_Low']
day_all_columsn = ['Date', 'Day_Open', 'Day_High', 'Day_Low', 'Day_Close', 'Day_RSI_EMA']
# data.iloc[-2]

# Add Signals Columns

In [5]:
def Add_Signals(dfnew):
    df = dfnew.copy()
    df['Signal_Day_RSI'] = np.select([df['Day_RSI_EMA'] <= 30, df['Day_RSI_EMA'] >= 70],['CE', 'PE'],default='0')
    
    df['Signal_RSI_CE'] = np.where(df['RSI_EMA'] <= 30, 'CE', '0')
    df['Signal_RSI_PE'] = np.where(df['RSI_EMA'] >= 70, 'PE', '0')


    ce_5ema = (df['High'].shift(1) < df['5EMA'].shift(1)) & (df['Close'] >= df['5EMA'])
    pe_5ema = (df['Low'].shift(1) > df['5EMA'].shift(1)) & (df['Close'] <= df['5EMA'])
    
    df['Signal_5EMA'] = np.select([ce_5ema, pe_5ema],['CE', 'PE'], default='NONE')

    ce_ema_crossover = (df['5EMA'] > df['15EMA']) & (df['5EMA'].shift(1) <= df['15EMA'].shift(1))
    pe_ema_crossover = (df['5EMA'] < df['15EMA']) & (df['5EMA'].shift(1) >= df['15EMA'].shift(1))
    
    df['Signal_EMA_CROSSOVER'] = np.select([ce_ema_crossover, pe_ema_crossover],['CE', 'PE'], default='0')
    
    df['SL_Price'] = list(zip(df['High'].shift(1) + 10, df['Low'].shift(1) - 10))
    
    # df.drop(columns = all_columsn,inplace= True)
    return df
newdata = Add_Signals(data)
newdata.iloc[-1]

Day_Open                             595.05
Day_High                              611.3
Day_Low                              595.05
Day_Close                            608.25
Day_RSI_EMA                            74.9
Day_Trend                           Uptrend
Open                                  607.6
High                                  608.2
Low                                  607.15
Close                                 608.0
5EMA                                 608.24
15EMA                                608.39
Candle                                Green
Prev_5EMA                            608.35
Prev_15EMA                           608.45
RSI                                   40.99
RSI_EMA                               49.08
Trend                              Sideways
First_Candle_High                     599.5
First_Candle_Low                     595.95
DateTime                2023-12-05 15:25:00
Signal_Day_RSI                           PE
Signal_RSI_CE                   

# ENTRY LOGIC

In [6]:
def CE_ENTRY_LOGIC(row):
    # status = row['Signal_5EMA'] == "CE" 
    status = row['Signal_EMA_CROSSOVER'] == "CE"
    
    return status
    
def PE_ENTRY_LOGIC(row):
    # status = row['Signal_5EMA'] == "PE" 
    status = row['Signal_EMA_CROSSOVER'] == "PE"
    return status

# STOPLOSS AND TARGET AND TRAILING

In [7]:
def GET_STOPLOSS_PRICE(row,status):
    N= 5 # Variance 
    
    if status == "CE":
        data1 = row["SL_Price"][1] -N
        data2 = row['Close'] - (row['Close']*0.3)# Recomded for index onky
    if status == "PE":
        data1 = row["SL_Price"][0] + N
        data2 = row['Close'] + (row['Close']*0.3)# Recomded for index onky
    return abs(data1)
    
def GET_TARGET_PRICE(row,status):
    N= 2 # Multiplate here 1:5 Risk Reword
    
    if status == "CE":
        data1 = N * (row['Close'] - row["SL_Price"][1] ) + row['Close']
        data2 = row['Close'] + (row['Close']*0.5) # Recomded for index onky
    if status == "PE":
        data1 = N * (row["SL_Price"][0] - row['Close']) - row['Close']
        data2 = row['Close'] - (row['Close']*0.5) # Recomded for index onky
    return abs(data1)
    

# LOGIC

In [8]:
CE_POSITION = False
CE_ENTRY_PRICE = 0
CE_ENTRY_DATETIME = None
CE_STOPLOSS = 0
CE_TARGET = 0
n = 0
m = 0
PE_POSITION = False
PE_ENTRY_PRICE = 0
PE_ENTRY_DATETIME = None
PE_STOPLOSS = 0
PE_TARGET = 0

def Main_Logic(row):
    global CE_POSITION, CE_ENTRY_PRICE, CE_STOPLOSS, CE_TARGET, CE_ENTRY_DATETIME ,n
    global PE_POSITION, PE_ENTRY_PRICE, PE_STOPLOSS, PE_TARGET, PE_ENTRY_DATETIME ,m

    Current_Price = row['Close']
    strick_price = int(round(row['Close'], -2))
    
    # -------------------------------------------------------------[CE]-----------------------------------------------------------------------------------
    if CE_ENTRY_LOGIC(row):
        # ENTRY
        if not CE_POSITION:  
            n+=1
            # Declaration
            CE_POSITION = True
            CE_ENTRY_PRICE = Current_Price
            CE_ENTRY_DATETIME = row['DateTime']
            CE_STOPLOSS = GET_STOPLOSS_PRICE(row,"CE")                
            CE_TARGET = GET_TARGET_PRICE(row ,'CE')
            return
            
    # EXIT SL
    if CE_POSITION and Current_Price <= CE_STOPLOSS:  
        Current_PNL = Current_Price - CE_ENTRY_PRICE
        
        algo_name = "5 EMA"
        symbol = "CE"
        entry_Strick_price = round(CE_ENTRY_PRICE, -2)
        entry_position = CE_ENTRY_PRICE
        entry_datetime = CE_ENTRY_DATETIME
        sl = CE_STOPLOSS
        sl_pr = abs((CE_STOPLOSS - CE_ENTRY_PRICE) / CE_ENTRY_PRICE * 100)
        target = CE_TARGET
        target_pr = abs((CE_TARGET - CE_ENTRY_PRICE) / CE_ENTRY_PRICE * 100)
        exit_position = Current_Price
        exit_datetime = row['DateTime']
        pnl = Current_PNL
        exit_status = "SL"
        Save_Data_in_Sheet(algo_name, symbol, entry_Strick_price, entry_position, entry_datetime, sl,sl_pr,target, target_pr,exit_position, exit_datetime, pnl, exit_status)

        # Declaration
        CE_POSITION = False
        CE_STOPLOSS = 0
        CE_TARGET = 0
        CE_ENTRY_PRICE = 0

    # EXIT TARGET
    if CE_POSITION and Current_Price >= CE_TARGET:
        Current_PNL = Current_Price - CE_ENTRY_PRICE 
        
        algo_name = "5 EMA"
        symbol = "CE"
        entry_Strick_price = round(CE_ENTRY_PRICE, -2)
        entry_position = CE_ENTRY_PRICE
        entry_datetime = CE_ENTRY_DATETIME
        sl = CE_STOPLOSS
        sl_pr = abs((CE_STOPLOSS - CE_ENTRY_PRICE) / CE_ENTRY_PRICE * 100)
        target = CE_TARGET
        target_pr = abs((CE_TARGET - CE_ENTRY_PRICE) / CE_ENTRY_PRICE * 100)
        exit_position = Current_Price
        exit_datetime = row['DateTime']
        pnl = Current_PNL
        exit_status = "TARGET"
        Save_Data_in_Sheet(algo_name, symbol, entry_Strick_price, entry_position, entry_datetime, sl,sl_pr,target, target_pr,exit_position, exit_datetime, pnl, exit_status)

        # Declaration
        CE_POSITION = False
        CE_STOPLOSS = 0
        CE_TARGET = 0
        CE_ENTRY_PRICE = 0

    # -------------------------------------------------------------[PE]-----------------------------------------------------------------------------------
    if PE_ENTRY_LOGIC(row):
        strick_price = int(round(row['Close'], -2))
        if not PE_POSITION: 
            # Declaration
            m+=1
            
            PE_POSITION = True
            PE_ENTRY_PRICE = Current_Price
            PE_ENTRY_DATETIME = row['DateTime']
            PE_STOPLOSS = GET_STOPLOSS_PRICE(row ,'PE')
            PE_TARGET = GET_TARGET_PRICE(row ,'PE')
            
            return
            
    # EXIT SL
    if PE_POSITION and Current_Price >= PE_STOPLOSS:  
        Current_PNL = PE_ENTRY_PRICE - Current_Price        
        
        algo_name = "5 EMA"
        symbol = "PE"
        entry_Strick_price = round(PE_ENTRY_PRICE, -2)
        entry_position = PE_ENTRY_PRICE
        entry_datetime = PE_ENTRY_DATETIME
        sl = PE_STOPLOSS
        sl_pr = abs((PE_STOPLOSS - PE_ENTRY_PRICE) / PE_ENTRY_PRICE * 100)
        target = PE_TARGET
        target_pr = abs((PE_TARGET - PE_ENTRY_PRICE) / PE_ENTRY_PRICE * 100)
        exit_position = Current_Price
        exit_datetime = row['DateTime']
        pnl = Current_PNL
        exit_status = "SL"
        
        Save_Data_in_Sheet(algo_name, symbol, entry_Strick_price, entry_position, entry_datetime, sl,sl_pr,target, target_pr,exit_position, exit_datetime, pnl, exit_status)

        # Declaration
        PE_POSITION = False
        PE_STOPLOSS = 0
        PE_TARGET = 0
        PE_ENTRY_PRICE = 0
            
    # EXIT TARGET
    if PE_POSITION and Current_Price <= PE_TARGET:
        Current_PNL = PE_ENTRY_PRICE - Current_Price 
        
        algo_name = "5 EMA"
        symbol = "PE"
        entry_Strick_price = round(PE_ENTRY_PRICE, -2)
        entry_position = PE_ENTRY_PRICE
        entry_datetime = PE_ENTRY_DATETIME
        sl = PE_STOPLOSS
        sl_pr = abs((PE_STOPLOSS - PE_ENTRY_PRICE) / PE_ENTRY_PRICE * 100)
        target = PE_TARGET
        target_pr = abs((PE_TARGET - PE_ENTRY_PRICE) / PE_ENTRY_PRICE * 100)
        exit_position = Current_Price
        exit_datetime = row['DateTime']
        pnl = Current_PNL
        exit_status = "TARGET"

        Save_Data_in_Sheet(algo_name, symbol, entry_Strick_price, entry_position, entry_datetime, sl,sl_pr,target, target_pr,exit_position, exit_datetime, pnl, exit_status)
        
        # Declaration
        PE_POSITION = False
        PE_STOPLOSS = 0
        PE_TARGET = 0
        PE_ENTRY_PRICE = 0

newdata.apply(Main_Logic,axis=1)
print("CE TRED :",n)
print("PE TRED :",m)

CE TRED : 8
PE TRED : 7


In [9]:
df = pd.read_excel("ALGO LOGS.xlsx")

In [10]:
# df[df['Exit Status'] == "TARGET"]

In [11]:
# df[df['Exit Status'] == "SL"]

In [12]:
total_tred = df.shape[0]
total_tred

13

In [13]:
total_SL_tred = df[df['Exit Status'] == "SL"].shape[0]
total_TARGET_tred = df[df['Exit Status'] == "TARGET"].shape[0]

In [14]:
print("TOTAL TRED:",total_tred)
print("SL COUNT :",total_SL_tred)
print("TARGET COUNT :",total_TARGET_tred)
print("TOTAL PNL:",round(df['P&L'].sum(),2))


TOTAL TRED: 13
SL COUNT : 8
TARGET COUNT : 5
TOTAL PNL: -9.25


In [15]:
df

,Algo Name,SYMBOL,Entry Strick Price,Entry Position,Entry Date Time,SL,SL(%),TARGET,TARGET(%),Exit Position,Exit Date Time,P&L,Exit Status,Tred Time
0,5 EMA,PE,600,571.75,2023-09-06 10:55:00,588.50,2.9,548.25,4.1,588.75,2023-09-11 09:25:00,-17.00,SL,7110
1,5 EMA,CE,600,573.55,2023-09-06 10:45:00,557.10,2.9,596.45,4.0,596.95,2023-09-13 13:45:00,23.40,TARGET,10260
2,5 EMA,PE,600,588.65,2023-09-11 11:05:00,604.80,2.7,566.35,3.8,604.95,2023-09-18 09:40:00,-16.30,SL,9995
3,5 EMA,CE,600,599.40,2023-09-14 10:55:00,583.45,2.7,621.30,3.7,582.35,2023-10-09 09:25:00,-17.05,SL,35910
4,5 EMA,PE,600,605.55,2023-09-18 12:00:00,621.80,2.7,583.05,3.7,582.35,2023-10-09 09:25:00,23.20,TARGET,30085
5,5 EMA,CE,600,584.10,2023-10-09 14:20:00,567.75,2.8,606.80,3.9,567.35,2023-10-19 09:15:00,-16.75,SL,14095
6,5 EMA,PE,600,586.55,2023-10-10 11:00:00,602.05,2.6,565.55,3.6,565.30,2023-10-20 11:45:00,21.25,TARGET,14445
7,5 EMA,CE,600,570.80,2023-10-19 11:15:00,554.80,2.8,592.80,3.9,554.30,2023-10-23 15:00:00,-16.50,SL,5985
8,5 EMA,PE,600,563.40,2023-10-20 14:35:00,578.80,2.7,542.60,3.7,578.80,2023-11-03 15:20:00,-15.40,SL,20205
9,5 EMA,CE,600,557.90,2023-10-25 09:35:00,538.80,3.4,586.10,5.1,586.25,2023-11-16 09:35:00,28.35,TARGET,31680


In [16]:
round(2.683511,1)

2.7

In [17]:
workbook